<a href="https://colab.research.google.com/github/joyceful-lee/Grid-World/blob/main/Grid_World.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
import numpy as np

teamId = 1308
worldId = 7
url = "https://www.notexponential.com/aip2pgaming/api/rl/gw.php"


payload={}
headers = {
  'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36',
  'x-api-key': '68e70f41323453cbf356',
  'userid': '1109',
  'Content-Type': 'application/x-www-form-urlencoded'}

environment_rows = 40
environment_columns = 40

q_values = np.zeros((environment_rows, environment_columns, 4))
actions = ['N', 'S', 'E', 'W']
current_reward = -0.1
newState= None
x = 0
y = 0

#teamid 1330
def create_team(name):
  url = "https://www.notexponential.com/aip2pgaming/api/index.php"
  payload = {'type': 'team',
        'name': f'{name}',
        }
  files = [

    ]
  response = requests.request("POST", url, headers=headers, data=payload, files=files)
  print(response.text)

#create_team('prj4')

def add_meb(teamId):
  url = "https://www.notexponential.com/aip2pgaming/api/index.php"
  payload = {'type': 'member',
        'teamId': f'{teamId}',
        'userid': '1109'
        }
  files = [

    ]
  response = requests.request("POST", url, headers=headers, data=payload, files=files)
  print(response.text)
#add_meb(1308)
def get_runs(teamId, count):
  query_url = "https://www.notexponential.com/aip2pgaming/api/rl/score.php?type=runs&teamId={teamId}&count={count}"
  response = requests.request("GET", query_url, headers=headers, data=payload)
  data = json.loads(response.text)
  print(data)
#get_runs(1330, 0)

def getMyTeam():
    query_url = "https://www.notexponential.com/aip2pgaming/api/index.php?type=myTeams"
    response = requests.request("GET", query_url, headers=headers, data=payload)
    team_data = json.loads(response.text)
    team = team_data["myTeams"]
    print(f"Teams:{team}")
    return team
#getMyTeam()

def getMembers(teamId):
    query_url = "https://www.notexponential.com/aip2pgaming/api/index.php?type=team&teamId={teamId}"
    response = requests.request("GET", query_url, headers=headers, data=payload)
    print(response.text)
# getMembers(1308)

def reset_team(teamId):
  query_url = f"https://www.notexponential.com/aip2pgaming/api/rl/reset.php?teamId={teamId}&otp=5712768807"
  response = requests.request("GET", query_url, headers=headers, data=payload)
  print(response.text)
# reset_team(1308)

def enter_world(teamId, worldId):
  payload = {'type': 'enter',
        'teamId': f'{teamId}',
        'worldId': f'{worldId}'}
  files = [

    ]
  response = requests.request("POST", url, headers=headers, data=payload, files=files)
  data = json.loads(response.text)
  print(data)
#enter_world(1308, 0)

def get_location(teamId):
  query_url = "https://www.notexponential.com/aip2pgaming/api/rl/gw.php?type=location&teamId={teamId}"
  response = requests.request("GET", query_url, headers=headers, data=payload)
  print(response.text)
#get_location(1308)

def get_state(teamId, move, worldId, x, y):
  global current_reward

  print(f"Next Move: {move}")
  payload = {'type': 'move',
        'teamId': f'{teamId}',
          'move': move,
        'worldId': f'{worldId}'}
  files = [

    ]
  response = requests.request("POST", url, headers=headers, data=payload, files=files)
  data = json.loads(response.text)
  print(data)
  reward = data['reward']
  position = data['newState']
  # print(position)
  # print(reward)
  current_reward = reward
  current_pos = position

  if(position!= None):
    x=int(position['x'])
    y=int(position['y'])
    # print(x)
    # print(y)
  """else:
    if(move=="N"):
      y +=1
    if(move=="S"):
      y -=1
    if(move=="E"):
      x +=1
    if(move=="W"):
      x -=1"""
  return x,y,reward

#get_runs(1309, 1)
#enter_world(1308,0)
#get_location(1307)
#enter_world(1309, 1)
#get_state(1308, "N", 0,0,39)


Win state reward is 10000.

Lose state reward is -10000.

The normal reward is -0.1


Reward:10000

{'code': 'OK', 'worldId': 0, 'runId': '6237', 'reward': 10000, 'scoreIncrement': 0, 'newState': None}


{'x': 0, 'y': '39'}+N
{'x': 39, 'y': '39'}+N
{'x': 19, 'y': '19'}+N

Reward:-10000

{'code': 'OK', 'worldId': 0, 'runId': '6537', 'reward': -10000, 'scoreIncrement': -1094.19, 'newState': None}

(1, 19, -10000)
(0,19,-10000)
(19,1,-10000)








In [ ]:
#check whether the state is a terminal state with reward feedback
def is_terminal(current_reward):
  #if the reward for this location is -0.1, then it is not a terminal state
  if current_reward < 10 and current_reward > -10:
    print('false', current_reward)
    return False
  else:
    print('true', current_reward)
    return True

#Select the move action
def get_action(x,y,epsilon):
    if np.random.random() < epsilon:
      return np.argmax(q_values[x, y])
    else:
      return np.random.randint(4)


#get the position and reward info with the determined action
#return x,y,reward
def get_pos_and_reward(action,x,y):
  if actions[action]=="N":
    return get_state(teamId,"N",worldId,x,y)
  if actions[action]=="S":
    return get_state(teamId,"S",worldId,x,y)
  if actions[action]=="E":
    return get_state(teamId,"E",worldId,x,y)
  if actions[action]=="W":
    return get_state(teamId,"W",worldId,x,y)




In [ ]:
import matplotlib.pyplot as plt

In [ ]:
#Q-learning parameters
def explore_world(world = 8, epoch = 100, current_reward=-0.1, discount_factor = 0.9, learning_rate = 0.9,epsilon = 0.1):
  ALL_q_values = []

  #run through 1000 training episodes
  for episode in range(epoch):
    print(f"********** EPOCH: {episode+1} **********")
    #get the starting location for this episode
    enter_world(1308, world)
    x, y = 0,0
    current_reward=-0.1

    learning_rate = learning_rate*discount_factor**(episode//100)
    epsilon = min(0.9, epsilon*1.1**(episode//epoch))
    # Exploration
    while not  is_terminal(current_reward):
      # select an action
      action = get_action(x, y, epsilon)

      #store the current position
      current_x, current_y = x, y
      x, y, current_reward = get_pos_and_reward(action,current_x, current_y)

      current_q_value = q_values[current_x, current_y, action]
      temporal_difference = current_reward + (discount_factor * np.max(q_values[x, y])) - current_q_value

      #update the q-value
      new_q_value = current_q_value + (learning_rate * temporal_difference)
      q_values[current_x, current_y, action] = new_q_value

    ALL_q_values.append(q_values)

  print('Complete learning.')
  return ALL_q_values

# reset_team(1325)
# explore_world(world=0, epoch=10)
#explore_world(world=1, epoch=1)
#explore_world(world=2, epoch=1)
# explore_world(world=3, epoch=10)
# explore_world(world=4, epoch=100)
# explore_world(world=5, epoch=5)
explore_world(world=7, epoch=5)
#explore_world(world=7, epoch=5)
#xplore_world(world=8, epoch=100)
# explore_world(world=9, epoch=5)
# explore_world(world=10, epoch=5)



********** EPOCH: 1 **********
{'code': 'OK', 'worldId': 7, 'runId': 31584, 'state': '0:0'}
false -0.1
Next Move: S
{'code': 'OK', 'worldId': 7, 'runId': '31584', 'reward': -0.1, 'scoreIncrement': -0.1, 'newState': {'x': '0', 'y': 0}}
false -0.1
Next Move: N
{'code': 'OK', 'worldId': 7, 'runId': '31584', 'reward': -0.1, 'scoreIncrement': -0.09, 'newState': {'x': 1, 'y': '0'}}
false -0.1
Next Move: N
{'code': 'OK', 'worldId': 7, 'runId': '31584', 'reward': -0.1, 'scoreIncrement': -0.08, 'newState': {'x': 0, 'y': '0'}}
false -0.1
Next Move: N
{'code': 'OK', 'worldId': 7, 'runId': '31584', 'reward': -0.1, 'scoreIncrement': -0.07, 'newState': {'x': '0', 'y': 1}}
false -0.1
Next Move: W
{'code': 'OK', 'worldId': 7, 'runId': '31584', 'reward': -0.1, 'scoreIncrement': -0.07, 'newState': {'x': '0', 'y': 2}}
false -0.1
Next Move: S
{'code': 'OK', 'worldId': 7, 'runId': '31584', 'reward': -0.1, 'scoreIncrement': -0.06, 'newState': {'x': '0', 'y': 1}}
false -0.1
Next Move: E
{'code': 'OK', 'world

[array([[[-0.33479861, -0.39074495, -0.28251408, -0.34499443],
         [-0.33211111, -0.33701911, -0.26932282, -0.38955997],
         [-0.25556978, -0.32497832, -0.26297353, -0.26712456],
         ...,
         [ 0.        ,  0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ,  0.        ]],
 
        [[-0.11673076, -0.33344163, -0.2179351 , -0.28208073],
         [-0.26227192, -0.26856805, -0.1969568 , -0.34072597],
         [-0.32309535, -0.32688752, -0.24813639, -0.32443922],
         ...,
         [ 0.        ,  0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        ,  0.        ]],
 
        [[-0.18713683, -0.32871064, -0.18900819, -0.33419478],
         [-0.26094568, -0.26222022, -0.238239  , -0.20092286],
         [-0.25545262, -0.24727231, -0.26143952, -0.23907847],
         ...,
       

In [ ]:

#play in the world with the trained Q-value table
def play():
    x, y = 0,0
    current_reward=-0.1
    epsilon = 0.1
    epsilon = min(0.9, epsilon*1.1**(1))
    path = []
    path.append([x, y])
    #keep moving until it is terminal
    while not is_terminal(current_reward):
      #get the best action
      action = get_action(x, y,epsilon)
      #move to the new state with the best action
      x, y, current_reward = get_pos_and_reward(action,x,y)
      #add the new position to path
      path.append([x, y])
    return path


enter_world(1308,6)
play_path = play()

# get_state(1308,"N",0,39,39)


{'code': 'OK', 'worldId': 6, 'runId': 31589, 'state': '0:0'}
false -0.1
Next Move: E
{'code': 'FAIL', 'message': 'No run is open for this team and world.'}


KeyError: ignored

In [ ]:
print(play_path)

In [ ]:
width, height, dir = q_values.shape

In [ ]:
for i in range(height-1, 0, -1):
  for j in range(width-1, 0, -1):
    print(q_values[j][i], end=' ')
  print('\n')

In [ ]:
disp_q_vals = np.zeros(q_values.shape[:2])

In [ ]:
for i in range(q_values.shape[0]):
  for j in range(q_values.shape[1]):
    n = np.argmax(abs(q_values[i][j]))
    # print(m)
    m = q_values[i][j][n]
    if abs(m) < 1:
      disp_q_vals[i][j] = abs(m)
    else:
      # disp_q_vals[i][j] = 1
      if m < 0:
        print(f"Penalty: {i},{j},{m}")
        disp_q_vals[i][j] = -1
      else:
        print(f"Reward Exit: {i},{j},{m}")
        disp_q_vals[i][j] = 1

plt.figure(figsize=(15,15))
plt.imshow(disp_q_vals)
plt.gca().invert_yaxis()
plt.colorbar(orientation='vertical')


In [ ]:
for i in range(q_values.shape[0]):
  for j in range(q_values.shape[1]):
    n = np.argmax(abs(q_values[i][j]))
    # print(m)
    m = q_values[i][j][n]
    if abs(m) < 1:
      disp_q_vals[i][j] = abs(m)
    else:
      disp_q_vals[i][j] = 1
      # if m < 0:
      #   disp_q_vals[i][j] = -1
      # else:
      #   disp_q_vals[i][j] = 1

plt.figure(figsize=(15,15))
plt.imshow(disp_q_vals)
plt.gca().invert_yaxis()
plt.colorbar(orientation='vertical')